In [4]:
#from functions import Standings
import pandas as pd
import numpy as np
import time

### Create csv

In [4]:
# Create a dataframe with all schedules from 1979-80 to 2022-23
years = np.arange(1979, 2024)
dfs = []
for year in years :

    print(f'Fetching standings for season {year-1}-{year} ...', end = '\r')
    dfs.append(Standings(year))
    time.sleep(3)

data = pd.concat(dfs).reset_index(drop = True)
data.to_csv('data/Schedules_1979-2023.csv', index = None)

KeyboardInterrupt: 

In [4]:
target = pd.read_csv('data/Standings_1979-2023.csv')[['Tm', 'Year', 'W/L%']]
target.to_csv('data/target_1979-2023.csv', index = None)

## Create team strength features

In [6]:
data = pd.read_csv('Standings_1979-2023.csv')
data.head()

,Team,Tm,Year,W,L,W/L%,PS/G,PA/G,SRS
0,Washington Bullets,WSB,1979,54,28,0.659,114.9,109.9,4.75
1,Seattle SuperSonics,SEA,1979,52,30,0.634,106.6,103.9,2.69
2,Phoenix Suns,PHO,1979,50,32,0.610,115.4,111.7,3.55
3,San Antonio Spurs,SAS,1979,48,34,0.585,119.3,114.1,4.97
4,Kansas City Kings,KCK,1979,48,34,0.585,113.1,110.2,2.73


### Last wins, Net Rating (=nombre de points - nbr de points encaissés)

In [101]:
# Returns dictionary where each team has its own list of dictionaries containing the year, win-loss-%, net-rating and SRS
def wins_rating(data,number_years=5):
    df_5_years = data[(data['Year'] >= 2023-number_years) & (data['Year'] <= 2023)]
    win_loss = {team: [] for team in df_5_years['Tm'].unique()}

    for _,row in df_5_years.iterrows():
        win_loss[row['Tm']].append({'Year': row['Year'], 'WinLoss': row['W/L%'],'NetRating': row['PS/G']-row['PA/G'], 'SRS': row['SRS']})
        #print(win_loss[team])
    return win_loss

In [103]:
teams_data = wins_rating(data)

# Create team_data.csv where each row corresponds to a team and contains columns for each year's win-loss-%, net rating, and SRS
def teams_csv(teams_data):
    # Transforming the data
    rows = []
    for team, years_data in teams_data.items():
        row = {'Team': team}
        for year_data in years_data:
            year = year_data['Year']
            row[f'WinLoss_{year}'] = year_data['WinLoss']
            row[f'NetRating_{year}'] = year_data['NetRating']
            row[f'SRS_{year}'] = year_data['SRS']
        rows.append(row)

    df_teams = pd.DataFrame(rows)

    # Fill NaN values if any year data is missing for a team
    df_teams.fillna('N/A', inplace=True)

    # Export to CSV
    df_teams.to_csv('team_data.csv', index=False)

teams_csv(teams_data)